In [1]:
# Instructions here: https://algotrading101.com/learn/alpaca-trading-api-guide/

import os
import pandas as pd
import datetime as dt
from pytz import timezone
import alpaca_trade_api as tradeapi

# authentication and connection details
# store your Alpaca API key and secret in your environment
api_key = os.environ.get('alpaca_api_key')
api_secret = os.environ.get('alpaca_api_secret')
base_url = 'https://api.alpaca.markets'

# for polygon connection
os.environ['APCA_API_KEY_ID']=api_key
os.environ['APCA_API_SECRET_KEY']=api_secret

# instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

# obtain account information
account = api.get_account()

In [2]:
# helper functions to pull data from polygon
def get_tick_data(symbol='SPY',date='2018-09-19',start='09:30:00',ticks=10000):
    full_date = date+" "+start
    st = dt.datetime.strptime(full_date, '%Y-%m-%d %H:%M:%S')
    st = timezone('US/Eastern').localize(st)
    st = int(st.timestamp())*1000
    trades = tradeapi.REST().polygon.historic_trades_v2(symbol=symbol, date=date, limit=ticks)
    trades.df.reset_index(level=0, inplace=True)
    #convert exchange numeric codes to names for readability
    exchanges = tradeapi.REST().polygon.exchanges()
    ex_lst = [[e.id,e.name,e.type] for e in exchanges]
    dfe = pd.DataFrame(ex_lst,columns=['exchange','exch','excode'])
    trades.df['exchange'] = trades.df['exchange'].astype(int)
    df = pd.merge(trades.df,dfe,how='left',on='exchange')
    df = df[df.exchange!=0]
    df.drop('exchange', axis=1, inplace=True)
    return df

def get_quote_data(symbol='SPY',date='2018-09-19',start='09:30:00',ticks=10000,cond=False):
    full_date = date+" "+start
    st = dt.datetime.strptime(full_date, '%Y-%m-%d %H:%M:%S')
    st = timezone('US/Eastern').localize(st)
    st = int(st.timestamp())*1000
    trades = tradeapi.REST().polygon.historic_quotes_v2(symbol=symbol, date=date, limit=ticks)
    trades.df.reset_index(level=0, inplace=True)
    return trades.df

In [3]:
## Pull daily data for AAPL
aapl = api.get_barset('AAPL', 'day')
aapl = aapl.df
aapl.head()

AAPL                                     
                               open      high     low    close    volume
time                                                                    
2020-07-22 00:00:00-04:00  386.7700  391.9000  386.41  389.090  18899399
2020-07-23 00:00:00-04:00  387.9935  388.3100  369.64  371.560  25747322
2020-07-24 00:00:00-04:00  363.9500  371.8800  356.58  370.420  43883515
2020-07-27 00:00:00-04:00  374.8400  379.6200  373.92  379.215  26920001
2020-07-28 00:00:00-04:00  377.4700  378.1986  372.99  373.040  23023333

In [4]:
## Pull 15min data for TSLA
tsla = api.get_barset('TSLA', '15Min')
tsla = tsla.df
tsla.head()

TSLA                                 
                              open     high     low   close  volume
time                                                               
2020-12-08 11:00:00-05:00  627.965  628.030  624.51  625.83   36787
2020-12-08 11:15:00-05:00  625.980  627.540  624.55  624.82   29404
2020-12-08 11:30:00-05:00  624.680  625.515  620.96  624.61   50546
2020-12-08 11:45:00-05:00  624.700  629.250  623.80  628.74  159956
2020-12-08 12:00:00-05:00  629.000  630.140  627.51  628.52   34133

In [5]:
## Pull 1min data for MSFT
msft = api.get_barset('MSFT', '1Min')
msft = msft.df
msft.head()

MSFT                               
                              open    high     low   close volume
time                                                             
2020-12-10 14:20:00-05:00  210.735  210.78  210.66  210.74   1937
2020-12-10 14:21:00-05:00  210.720  210.72  210.57  210.57   1606
2020-12-10 14:22:00-05:00  210.570  210.60  210.57  210.57   1696
2020-12-10 14:23:00-05:00  210.650  210.65  210.59  210.59    690
2020-12-10 14:24:00-05:00  210.550  210.55  210.55  210.55    239

In [6]:
## Pull last 1k bars of 1min data for TTD 
ttd = api.get_barset(symbols='TTD', timeframe='1Min', limit=1000)
ttd = ttd.df
ttd.head()

TTD                               
                             open    high     low   close volume
time                                                            
2020-11-30 12:33:00-05:00  879.72  879.72  879.72  879.72    101
2020-11-30 12:37:00-05:00  879.59  879.59  879.59  879.59    107
2020-11-30 12:41:00-05:00  880.73  880.73  880.73  880.73    134
2020-11-30 12:45:00-05:00  882.08  882.08  882.08  882.08    100
2020-11-30 12:47:00-05:00  882.69  882.69  882.69  882.69    208

In [7]:
## Pull last 1k bars of 1min data for GOOGL 
googl = api.get_barset(symbols='GOOGL', timeframe='1Min', start='2020-01-15T09:30:00-04:00', limit=1000)
googl = googl.df
googl.head()

GOOGL                                  
                              open     high      low    close volume
time                                                                
2020-12-04 11:41:00-05:00  1827.00  1827.00  1827.00  1827.00    100
2020-12-04 11:42:00-05:00  1826.25  1826.50  1826.25  1826.50    204
2020-12-04 11:43:00-05:00  1826.46  1826.46  1825.77  1825.78    784
2020-12-04 11:44:00-05:00  1825.48  1825.48  1825.48  1825.48    334
2020-12-04 11:45:00-05:00  1825.38  1825.38  1825.01  1825.01    564

In [8]:
# get SPY tick data
get_tick_data(date='2018-08-01').head()

,sip_timestamp,participant_timestamp,trf_timestamp,sequence_number,id,size,conditions,price,tape,exch,excode
0,2018-08-01 04:09:41.276901-04:00,1533110981276221440,NaN,13701,52983525027890,300,"[14, 12, 41]",281.37,1,NYSE Arca,exchange
1,2018-08-01 04:11:25.339299-04:00,1533111085338292496,NaN,13801,62879129944657,100,"[14, 12, 41]",281.40,1,Nasdaq,exchange
2,2018-08-01 04:12:03.076945-04:00,1533111123076040361,NaN,13901,62879129944675,400,"[14, 12, 41]",281.46,1,Nasdaq,exchange
3,2018-08-01 04:12:11.448132-04:00,1533111131447281976,NaN,14001,62879129944676,100,"[14, 12, 41]",281.50,1,Nasdaq,exchange
4,2018-08-01 04:12:19.338575-04:00,1533111139337746024,NaN,14101,62879129944686,100,"[14, 12, 41]",281.50,1,Nasdaq,exchange


In [9]:
# get SPY quote data
get_quote_data(date='2018-08-01').head()

,sip_timestamp,participant_timestamp,trf_timestamp,sequence_number,conditions,indicators,bid_price,bid_exchange,bid_size,ask_price,ask_exchange,ask_size,tape
0,2018-08-01 04:00:00.184891-04:00,1533110400184230655,NaN,8401,"[1, 81]",NaN,281.23,12,5,282.00,11,4,1
1,2018-08-01 04:00:41.624184-04:00,1533110441623709696,NaN,8801,"[1, 81]",NaN,281.30,11,5,281.53,12,5,1
2,2018-08-01 04:00:51.848616-04:00,1533110451848372480,NaN,9301,"[1, 81]",NaN,281.33,11,5,281.53,12,5,1
3,2018-08-01 04:00:51.849575-04:00,1533110451849319680,NaN,9501,"[1, 81]",NaN,281.33,12,5,281.53,12,5,1
4,2018-08-01 04:00:51.850724-04:00,1533110451850105952,NaN,10001,"[1, 81]",NaN,281.31,11,5,281.53,12,5,1
